<a href="https://colab.research.google.com/github/NikFloden/Art-Style-Transfer-Using-Neural-Networks/blob/main/Milestone_2_Understanding_What_Convolutional_Neural_Networks_Learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding What Convolutional Neural Networks Learn
**Objective**

*   Load the trained CNN and understand what exactly it’s learning.

**Importance to project**

*   Understand the abilities of the trained CNN model to understand not only the way to build better models but also the model’s weaknesses and strengths.

*  Learn how to view misclassified samples, which will help to understand that CNNs are not infallible and give some intuition into how they work.

* Learn to display the feature maps that are output by various convolution and pooling layers in a network, given certain input into your trained CNN.

**Workflow**


1.   Load the trained MNIST model from Milestone 1.
2.   Analyze the model’s performance by viewing the confusion matrix and  classification report.
  *   Closely examine the performance metrics of the trained CNN and identify its weaknesses.
  *  Look closely at the individual performance of each class and note the classes the CNN has trouble classifying correctly.
3.  Identify all samples wrongly classified by the CNN and display them.

Observe this data and intuitively think about why the CNN misclassified them.
4.  Extract the layer outputs and create the activation layer model so you can visualize the activations of each layer.

This requires you to give the CNN a certain input, such as an image from your test dataset.
5.  Display the output shape of the first layer activation and compare it with the output layer shape that can be seen from displaying the model summary.
6.  Show the output of the activation map of the fourth convolution filter or feature map in the first convolution layer.
7.  Display all the activations for the first three layers of your model (in other words–the two convolution layers and the max-pooling layer).




## 1.  Loading the Model

In [3]:
from tensorflow.keras.models import load_model
model = load_model("mnist_simple_cnn_10_Epochs.h5")

##2.  Analyzing the model's performance